In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    pipeline,
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
)
from langchain.llms import HuggingFacePipeline

# =============================
# 1. Load Model & Tokenizer
# =============================

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id)

# =============================
# 2. Prepare Model for QLoRA
# =============================

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, peft_config)

# =============================
# 3. Load and Format Dataset
# =============================

df = pd.read_csv("diabetes_dataset.csv")  # Ensure file is in the same directory
dataset = Dataset.from_pandas(df)

# Format input to match Alpaca-style training
def format_and_tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example.get('input', '')}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=256)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    format_and_tokenize,
    remove_columns=dataset.column_names,
    batched=False,
    load_from_cache_file=False
)

# =============================
# 4. Training Arguments
# =============================

training_args = TrainingArguments(
    output_dir="./tinyllama_diabetes_qlora",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=1,
    save_strategy="no",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# =============================
# 5. Train the Model
# =============================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

# =============================
# 6. Save the Model
# =============================

#trainer.save_model("./tinyllama_diabetes_qlora")
#tokenizer.save_pretrained("./tinyllama_diabetes_qlora")
from peft import PeftModel, PeftConfig

# First, save the PEFT adapter
model.save_pretrained("./tinyllama_diabetes_qlora_adapter")

# Load the adapter for merging
model = PeftModel.from_pretrained(model, "./tinyllama_diabetes_qlora_adapter")

# Merge adapters into the base model
model = model.merge_and_unload()

# Save full merged model
model.save_pretrained("./tinyllama_diabetes_qlora")
tokenizer.save_pretrained("./tinyllama_diabetes_qlora")
# =============================
# 7. Inference with LangChain
# =============================

pipe = pipeline(
    "text-generation",
    model="./tinyllama_diabetes_qlora",
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    device=0 if torch.cuda.is_available() else -1,
)

llm = HuggingFacePipeline(pipeline=pipe)

# =============================
# 8. Test the Model
# =============================

prompt = "### Instruction:\nWhat is insulin resistance?\n\n### Input:\n\n### Response:\n"
print("\n📜 Generated Output:\n")
print(llm(prompt))



c:\Users\HP\3_GenAI\4_Qlora_Finetuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 100/100 [00:00<00:00, 1492.19 examples/s]
C:\Users\HP\AppData\Local\Temp\ipykernel_11584\3531480175.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
c:\Users\HP\3_GenAI\4_Qlora_Finetuning\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device 

Step,Training Loss
1,2.681700
2,2.512900
3,2.507700
4,2.664300
5,2.466700
6,2.602500
7,2.540500
8,2.369700
9,2.265200
10,2.218800


ValueError: Can't find 'adapter_config.json' at './tinyllama_diabetes_qlora'

Loading adapter weights from ./tinyllama_diabetes_qlora led to missing keys in the model: model.layers.0.self_attn.q_proj.lora_A.default.weight, model.layers.0.self_attn.q_proj.lora_B.default.weight, model.layers.0.self_attn.v_proj.lora_A.default.weight, model.layers.0.self_attn.v_proj.lora_B.default.weight, model.layers.1.self_attn.q_proj.lora_A.default.weight, model.layers.1.self_attn.q_proj.lora_B.default.weight, model.layers.1.self_attn.v_proj.lora_A.default.weight, model.layers.1.self_attn.v_proj.lora_B.default.weight, model.layers.2.self_attn.q_proj.lora_A.default.weight, model.layers.2.self_attn.q_proj.lora_B.default.weight, model.layers.2.self_attn.v_proj.lora_A.default.weight, model.layers.2.self_attn.v_proj.lora_B.default.weight, model.layers.3.self_attn.q_proj.lora_A.default.weight, model.layers.3.self_attn.q_proj.lora_B.default.weight, model.layers.3.self_attn.v_proj.lora_A.default.weight, model.layers.3.self_attn.v_proj.lora_B.default.weight, model.layers.4.self_attn.q_pro


📜 Generated Output:

### Instruction:
What is insulin resistance?

### Input:

### Response:
Insulin resistance refers to the body's inability to respond to insulin, resulting in increased blood sugar levels and weight gain. It is a condition that affects many people with diabetes, and is typically caused by a combination of lifestyle factors, such as poor diet, sedentary lifestyle, and stress.
